# Vasculature simulation use-case

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()

In [ ]:
status = core.set_camera(
    orientation = [-0.188, 0.28, 0.036, 0.940],
    position = [1577, 2183, 2511],
    target = [333, 1279, 646])

In [ ]:
status = core.set_renderer(
    current='bio_explorer',
    subsampling=4, max_accum_frames=32)
params = core.BioExplorerRendererParams()
params.epsilon_factor = 1000.0
params.shadows = 1.0
params.soft_shadows = 1.0
status = core.set_renderer_params(params)

### 

### Load vasculature

In [ ]:
data_folder = '/home/favreau/Downloads/'
vasculature_file = data_folder + 'graph_with_endfeet.h5'
vasculature_model = be.add_vasculature(
    name='Vasculature', path=vasculature_file,
    use_sdf=False, load_capilarities=False
)

In [ ]:
be.set_vasculature_color_scheme(
    color_scheme=be.VASCULATURE_COLOR_SCHEME_SUBGRAPH,
    palette_name='rainbow'
)

## Simulation reports

In [ ]:
vasculature_info = be.get_vasculature_info()
print(vasculature_info) 

In [ ]:
from bioexplorer import TransferFunction
vasculature_model_id = vasculature_info['modelId']
tf = TransferFunction(
    bioexplorer=BioExplorer(url), name='Reds',
    model_id=vasculature_model_id, size=16, alpha=1.0
)

In [ ]:
''' Report file names and value ranges '''
reports = ['report_flows', 'report_pressures', 'report_radii']
reports_ranges = [[-0.5, 1.5], [0, 0.0015], [0, 45]]

''' Attach report to vasculature '''
report_id = 2
debug = True

report_filename = data_folder + reports[report_id] +'.h5'
be.set_vasculature_report(
    path=report_filename, debug=debug)
if debug:
    tf.set_range([0.0, 1.0])
else:
    tf.set_range(reports_ranges[report_id])

## Radius report

In [ ]:
import time
report_filename = data_folder + 'report_radii.h5'

core.set_renderer(
    current='bio_explorer',
    subsampling=1, max_accum_frames=1)

for frame in range(0, 360, 9):
    core.set_animation_parameters(current=frame)
    be.set_vasculature_radius_report(
        path=report_filename, frame=frame, amplitude=2.0, debug=True)
    core.set_renderer()
    time.sleep(0.1)